<a href="https://colab.research.google.com/github/pavelpryadokhin/image-metrics/blob/main/CLIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

In [ ]:
import numpy as np
import torch
from pkg_resources import packaging
import clip

print("Torch version:", torch.__version__)
clip.available_models()
model, preprocess = clip.load("ViT-B/32")
model.cuda().eval()
input_resolution = model.visual.input_resolution
context_length = model.context_length
vocab_size = model.vocab_size

print("Model parameters:", f"{np.sum([int(np.prod(p.shape)) for p in model.parameters()]):,}")
print("Input resolution:", input_resolution)
print("Context length:", context_length)
print("Vocab size:", vocab_size)
preprocess
clip.tokenize("Hello World!")


In [ ]:
import os
from PIL import Image
import numpy as np
from collections import OrderedDict
import torch

%matplotlib inline
%config InlineBackend.figure_format = 'retina'


In [ ]:
from google.colab import files
import os
import shutil

# Загрузка файла с локального компьютера
uploaded = files.upload()

folder_path = '/content/images/ChatGpt'
# folder_path = '/content/1'

# Укажите путь к загруженному файлу
file_path = list(uploaded.keys())
for file in file_path:
  try:shutil.move(file, folder_path)
  except: pass

In [ ]:
y='/content/images/ChatGpt'
print(y.split('/')[-1])

In [ ]:
import zipfile


# Укажите путь к загруженному архиву для распаковки
zip_path = '/content/Картинки.zip'
# Распаковка архива
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/images/')

In [ ]:

def solution(images_path):
  images_dict = {}
  for image_name in os.listdir(images_path):
      image_path = os.path.join(images_path, image_name)
      if os.path.isfile(image_path):
          key = int(image_name.replace(",", ".").split(".")[0])
          if key in images_dict:
              images_dict[key].append(image_path)
          else:
              images_dict[key] = [image_path]
  texts={1:"wrench", 2:"a wrench in the hands of a master who repairs a car engine",
        3: "a screwdriver", 4:"there are construction tools and a screwdriver on the table",
        5: "ladle furnaces in steel production",6: "ladle furnaces in steel production with a number on the ladle",
        7:"box on the conveyor belt", 8:"a box on a conveyor belt with a barcode"}


  def foo(images_dict,text):
    original_images = []
    images = []
    for path in images_dict:
      image = Image.open(path).convert("RGB")
      original_images.append(image)
      images.append(preprocess(image))
    image_input = torch.tensor(np.stack(images)).cuda()
    text_tokens = clip.tokenize(["This is " +  text]).cuda()
    with torch.no_grad():
      image_features = model.encode_image(image_input).float()
      text_features = model.encode_text(text_tokens).float()
    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = text_features.cpu().numpy() @ image_features.cpu().numpy().T

    return [np.median(similarity),np.mean(similarity), np.std(similarity),similarity.tolist()[0]]

  res=dict()

  for key in images_dict:
    res[key]=foo(images_dict[key],texts[key])
  return res


#   print(texts[key])
#   print(res[-1][-1])
#   print('медиана  ', round(res[-1][0],2))
#   print('среднее  ', round(res[-1][1],2))
#   print('ско  ', round(res[-1][2],2))
# print(res)


images_path = ['/content/images/ChatGpt','/content/images/DeepDream','/content/images/Kandinsky','/content/images/Midjourney','/content/images/Stable Diffusion','/content/images/dreamstudio','/content/images/shedevrum']
ans=dict()

for i in images_path:
  ans[i.split('/')[-1]]=solution(i)
print(ans)

In [ ]:
for res in ans:
  print(res)
  for val in sorted(ans[res]):
    print(round(ans[res][val][1],2))

ChatGpt
0.29
0.3
0.29
0.28
0.32
0.32
0.25
0.28
DeepDream
0.23
0.28
0.3
0.28
0.32
0.3
0.27
0.29
Kandinsky
0.29
0.31
0.31
0.29
0.33
0.33
0.24
0.26
Midjourney
0.26
0.31
0.3
0.27
0.32
0.32
0.26
0.29
Stable Diffusion
0.28
0.29
0.3
0.27
0.32
0.32
0.26
0.28
dreamstudio
0.3
0.3
0.29
0.28
0.32
0.33
0.25
0.28
shedevrum
0.27
0.29
0.31
0.27
0.32
0.32
0.24
0.27
